# *k*-Nearest Neighbours 
Hogeschool Utrecht (c) 2020

Tijmen Muller (tijmen.muller@hu.nl) en nabewerking Joost Vanstreels (joost.vanstreels@hu.nl)

In dit notebook vind je 2 opdrachten:
- <a href="#knn_opdracht1"> Opdracht 1 </a> laat eerst het voorbeeld zien uit het video-college. Vervolgens moet je zelf aan de slag. Onderaan deze opdracht worden hints gegeven, maar het is belangrijk dat je eerst zelf nadenkt!
- <a href="knn_opdracht2"> Opdracht 2 </a> maken we in het college.

<a id="knn_opdracht1"> </a>
## Opdracht 1
Het voorbeeld hieronder werd ook in het videocollege getoond. De score van dit baseline model is nog niet goed genoeg. Aan jullie de taak om dit model te verbeteren!

Daar zijn verschillende manieren voor. Probeer eerst zelf te bedenken welke verbeteringen je kunt doorvoeren op deze implementatie om de high-score omhoog te krikken. Als dat niet lukt, kun je onderaan een aantal hints bekijken:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix

## Data inlezen

In [ ]:
# inlezen tabel met fruit gegevens
fruits = pd.read_table('knn_fruit.txt')
print('Aantal meetwaarden {0:d}'.format(len(fruits)))
fruits.head()

In [ ]:
fruits.value_counts(["fruit_name"])

De kolommen zijn als volgt:
* `fruit_label` is een identificatienummer, overeenkomend met `fruit_name`
* `fruit_name` is het fruittype
* `fruit_subtype` is het subtype (bijvoorbeeld het soort appel)
* `mass` is het gewicht in grammen
* `width` is de breedte in cm
* `height` is de hoogte in cm
* `color_score` is een waarde uit het kleuren spectrum:
  * groen: 0.45-0.65
  * geel: 0.65-0.75
  * oranje: 0.75-0.85
  * rood: 0.85-1.00

In [ ]:
# maak een dictionary van fruit_label naar fruit_name
lookup_fruit_name = dict(zip(fruits['fruit_label'].unique(),fruits['fruit_name'].unique()))
lookup_fruit_name

## Aanpak volgens werkwijze `scikit-learn`

### 1. Kies het modeltype

We kiezen _k_-Nearest Neighbours (`KNeighborsClassifier`)

In [ ]:
knn = KNeighborsClassifier()

### 2. Organiseer de data

Onze feature matrix `X` bestaat uit de kolommen `mass`, `width` en `height`: dit zijn de kenmerken waar _vanuit_ we willen voorspellen.

Onze target vector `y` bestaat uit de kolom `fruit_label`: dit is het resultaat waar we _naartoe_ willen voorspellen (bij _k_-NN moet deze numeriek zijn, ookal is het een klasse).

In [ ]:
X = fruits[['mass','width','height']]
y = fruits['fruit_label']

### 3. Creëer een train- en validatieset

De methode `sklearn.model_selection.train_test_split()` deelt de feature matrix en de result vector gerandomiseerd op in een train- en een validatieset (ook wel: testset).

In [ ]:
# splits in train en test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

print(f'{len(X_train)} trainwaarden, {len(X_test)} testwaarden: {len(X_train)*100/len(fruits):.1f}%/{len(X_test)*100/len(fruits):.1f}%')

### 4. Kies de hyperparameters

In [ ]:
knn.set_params(n_neighbors = 5, weights = 'uniform')

### 5. Train het model

In [ ]:
knn.fit(X_train,y_train)

### 6. Valideer het model

Het model is te valideren met de validatieset. Het model heeft de validatieset nog niet gezien (alleen de trainset is gebruikt om het model te trainen). Het model kan de resultaten voorspellen op de _feature matrix_ van de validatieset. Door de uitkomsten (voorspeld resultaat) te vergelijken met de _target vector_ van de validatieset (de échte waarden) kunnen we zien hoe goed het model voorspeld.

We kunnen de methode `sklearn.metrics.accuracy_score()` gebruiken om de voorspelde waarde en de echte waarde te vergelijken, dit geeft een percentage 'goed voorspeld'. _k_-NN heeft ook zijn eigen methode `score()`, deze doet hetzelfde.

In [ ]:
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
knn.score(X_test,y_test)

53% is natuurlijk niet zo'n goede score. Om te bepalen _hoe_ goed (of slecht) de score is, is het goed om na te denken over een baseline. Als we heel naïef/dom zouden voorspellen, wat voor score zouden we dan behalen? In dit voorbeeld zouden we bijvoorbeeld kunnen kijken welk fruittype het vaakst voorkomt en elk stuk fruit in die klasse plaatsen (best dom, toch?).

In [ ]:
fruits.groupby('fruit_name').count()

'apple' en 'orange' komen het vaakst voor, laten we 'orange' kiezen. Deze heeft `fruit_label` = 3, dus in onze 'naieve' voorspelling is het resultaat _altijd_ een 3.

In [ ]:
y_naive = np.full(len(y_pred), 3)
print(f"Resultaatvector bij 'dom' voorspellen: {y_naive}\nBaseline score: {accuracy_score(y_test, y_naive)}")

Dus als we naief voorspellen, behalen we dezelfde score! Dat is niet best...

Een andere handige referentie is de score op de *trainset*. Dit zijn de waardes _waarop_ het model heeft getraind, dus hierop zou het model natuurlijk goed moeten scoren (het model heeft zich immers op die waarden gebaseerd).

In [ ]:
y_train_pred = knn.predict(X_train)
print(f"Score bij voorspellen op de trainset: {accuracy_score(y_train, y_train_pred)}")

Dat is een veel betere voorspellingsscore -- blijkbaar voorspelt het model op de trainingswaarden wél goed en op de validatiewaarden niet. In zo'n geval zou er sprake kunnen zijn van _overfitting_.

Nu willen we natuurlijk graag inzicht  _waarom_ het model niet goed scoort. Bij classificatie kunnen we daarvoor een confusion matrix gebruiken: deze vergelijkt de voorspelde waarde met de echte waarde. We kunnen dit doen met de methode `sklearn.metrics.confusion_matrix()` (geeft een NumPy `array` terug) en vervolgens visualiseren met bijvoorbeeld Seaborn.

In [ ]:
cm = confusion_matrix(y_test, y_pred)

# Maak van de array een pandas dataframe om te visualiseren
df_cm = pd.DataFrame(cm, 
                     index = [lookup_fruit_name[i+1] for i in range(4)], 
                     columns = [lookup_fruit_name[i+1] for i in range(4)])

print(cm)
df_cm

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5), dpi=100)

ax = sns.heatmap(df_cm, annot=True, cmap='Greens')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

ax.set_xlabel('voorspelde waarde')
ax.set_ylabel('echte waarde')

plt.show()

### 7. Voorspel nieuwe data

We kunnen nu het fruittype voorspellen van een nieuw meetpunt op basis van de kenmerken `mass`, `width` en `length` (met helaas maar een lage betrouwbaarheid, vanwege de lage voorspellingsscore). Bijvoorbeeld: wat voor fruittype is een nieuw stuk fruit met massa 150, breedte 6.5 en hoogte 7?

In [ ]:
fruit_feat = [150,6.5,7.0]
fruit_pred = knn.predict([fruit_feat])
print(fruit_pred)
print(f"Voorspelling: fruit_label = {fruit_pred[0]}, fruit_name = {lookup_fruit_name[fruit_pred[0]]}")

Als je niet beter zou weten, zou je klakkeloos aannemen dat dit stuk fruit een appel is. Maar als je kijkt naar de waarschijnlijkheden van de voorspelling, zie je dat het model niet zo zeker van z'n zaak is.

In [ ]:
fruit_pred_proba = knn.predict_proba([fruit_feat])
print(fruit_pred_proba)
print(f"Voorspelling: fruit_label_proba = {fruit_pred_proba[0,0]}, fruit_name = {lookup_fruit_name[1]}")
print(f"Voorspelling: fruit_label_proba = {fruit_pred_proba[0,1]}, fruit_name = {lookup_fruit_name[2]}")
print(f"Voorspelling: fruit_label_proba = {fruit_pred_proba[0,2]}, fruit_name = {lookup_fruit_name[3]}")
print(f"Voorspelling: fruit_label_proba = {fruit_pred_proba[0,3]}, fruit_name = {lookup_fruit_name[4]}")

## Oefening

Bedenk verschillende aanpassingen die een beter resultaat zouden kunnen opleveren. Voer een variant uit en verifieer het resultaat. Het is mogelijk om een score van 1.0 op deze testset te halen!

Merk op: probeer eerst zelf op basis van de theorie van de afgelopen colleges na te denken over acties die je uit kunt voeren om een model te verbeteren. Denk bijvoorbeeld aan de Supervised Learning workflow!

## SPOILER ALERT


##

##

##

##

##

## HIERONDER STAAN DE TIPS


##

##

##

##

##
## DENK EERST ZELF NA!

#### Verbetering 1. Hyperparameters aanpassen.
In het voorbeeld wordt _k_ = 5 gekozen. Controleer of er verbetering is als je een andere waarde voor _k_ kiest.

#### Verbetering 2. Features toevoegen.

In het voorbeeld worden alleen `mass`, `width`, `height` als feature gebruikt. Controleer of er verbetering is als je `color_score` toevoegt.

#### Verbetering 3. Normaliseren

In het voorbeeld zijn de features niet genormaliseerd. Controleer of er verbetering is als je de features normaliseert.

#### Verbetering 4. Hyperparameters nog verder aanpassen.
In het voorbeeld wordt met een uniforme gewicht gewerkt. Wat gebeurt er als je het gewicht laat afhangen van de distance?

Tip: speel ook nog even met de hoogte voor k

<a id="knn_opdracht2"></a>
## Opdracht 2

Tijdens het college gaan we met z'n allen kijken naar de bekende MNIST (Modified National Institute of Standards and Technology) data set. Deze data set zit standaard in sklearn.datasets en bevat afbeeldingen (van 8 x 8 = 64 pixels) van handgeschreven cijfers. Elke pixel heeft een integer waarde die de grijswaarde aangeeft van 0 (wit) tot 16 (zwart).

Ons doel is om met behulp van k-NN van een nieuw afbeelding te kunnen voorspellen welk cijfer er geschreven is.

#### Initialisatie
In de cell hieronder zetten we alle statements die packages inladen netjes bij elkaar. In de loop van het college zullen we hier nog een en ander aan toe moeten voegen.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors

#### Inlezen van de data
In sci kit learn (sklearn) is een submodule datasets. Data sets worden steeds als functie gegeven: door het uitvoeren van de functie krijg je je data en nog wat extra zaken, zoals een beschrijving.
Hieronder laden we de data alvast in.

In [ ]:
data = load_digits()

#### Verkenning van de data
Nu gaan we aan de slag om de data te verkennen. Bekijk eerst hoe het data object is opgebouwd dat load_digits() teruggeeft. Probeer dan te achterhalen hoeveel datapunten (afbeeldingen) we hebben, en hoe de cijfers relateren aan plaatjes van handgeschreven cijfers. Voor dit laatste een tip: matplotlib.pyplot heeft een commando 'imshow'.  

In [ ]:
print(str(data))

In [ ]:
print(data.DESCR)

In [ ]:
print(type(data.data))
print(type(data.target))
print(type(data.images))

In [ ]:
print(data.data.shape)
print(data.target.shape)
print(data.images.shape)

In [ ]:
plt.imshow(data.images[0], cmap = "Greys")

In [ ]:
f, axes = plt.subplots(8, 8, sharey=True, figsize=(12, 12))
for i in range(8):
    for j in range(8):
        axes[i,j].imshow(data.images[8*i+j], cmap = "Greys")

### Opdracht 2a
We willen niet alleen een classifier trainen, we willen ook weten hoe goed ie het doet. Splits daarom nu de data in een train set (80% van de observaties) en een test set (20% van de observaties). De train set gaan we gebruiken om het algoritme te trainen, de test set om te kijken wat de nauwkeurigheid van dit algoritme is.

**Let op**: denk eerst goed na wat de *features* zijn en wat de *target* is. In de cellen hierboven kun je dat zien en zo bepalen hoe de train- en testdata gemaakt kan worden. 

### Opdracht 2b
Nu gaan we onze classifier trainen met k-NN. Dat betekent het algoritme gaat zoeken naar plaatjes die *het dichtsbij* liggen. Voor twee plaatjes zal er voor alle pixels gekeken worden wat de afstand is en wordt de totale afstand berekend. De labels van de plaatjes waarvan de totale afstand het kleinste is, worden gebruikt om het label van het nieuwe plaatje te voorspellen.

Een belangrijke keuze hierbij is afstandsfunctie die we gebruiken om te bepalen of twee plaatjes op elkaar lijken. Wij zullen de (default) Minkowski afstand gebruiken. Als je niet weet wat de Minkowski afstand is, zoek dat dan even op. 

Wat betekent dit voor twee afbeeldingen? Een afbeelding bestaat feitelijk uit 8 vectoren van 8 cijfers: 8x8 pixels. Maar we hebben tot nu toe altijd gewerkt met tabellen uit .csv bestanden... Daarom gaan we gebruik maken van data.data en niet van data.images want in data.data staat alle pixels op een rij van 64 cijfers, eigenlijk als een soort tabel! Als je in de vorige stap images hebt gebruikt, dan moet je dit dus even wijzigen.

Enfin, de opdracht dus: voer nu het trainen van k-NN uit!

### Opdracht 2c
Bereken nu de accuracy van onze classifier op de test set. Welk percentage van de cijfers wordt goed voorspeld? Wat vind je van dit percentage?

Laten we als extra check ook even wat plaatjes bekijken uit onze test set en daarbij het voorspelde cijfer en het werkelijke cijfer. 

In [ ]:
f, axes = plt.subplots(4, 8, sharey=True, figsize=(8, 8))
for i in range(4):
    for j in range(8):
        axes[i,j].imshow(X_testd[8*i+j].reshape([8,8]), cmap = "Greys")
print('Voorspelde waarden: ', knn_pred[0:32])
print('Echte waarden:      ', y_testd[0:32])

### Extra opgaven
- Er zijn ook andere afstandsfuncties. Kijk wat de impact is op de accuracy.
- Hoe goed kun je classificeren als je alleen de onderste helft van de plaatjes mag gebruiken?
- Voor het eerste cijfer van de test set, wat zijn de 5 buren? Wat was de bijbehorende target van die buren?